In [1]:
import pandas as pd

##### Задание 1  
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
	- оценка 2 и меньше - низкий рейтинг
	- оценка 4 и меньше - средний рейтинг
	- оценка 4.5 и 5 - высокий рейтинг  
Результат классификации запишите в столбец *class*

In [3]:
# Формируем датафрейм с объединением базы фильмов и базы рейтингов
ratings_df = pd.read_csv('F:\\Program\\git_base\\GitHub\\MyRepo\\PYDA_07\\ml-latest-small\\ratings.csv', \
                         sep=',', header=0, encoding='UTF-8')
movie_df = pd.read_csv('F:\\Program\\git_base\\GitHub\\MyRepo\\PYDA_07\\ml-latest-small\\movies.csv', \
                         sep=',', header=0, encoding='UTF-8')
joined_df = movie_df.merge(ratings_df, on='movieId', how='left')
joined_df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,8.518667e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,9.386292e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,5.0,1.331380e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.0,9.979383e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,3.0,8.551901e+08


In [4]:
# Функция простановки рейтинга
def class_name(a):
    """определяем класс по рейтингу"""
    if a >= 4.5:
        return "Высокий рейтинг"
    elif a > 2.0:
        return "Средний рейтинг"
    elif a <= 2.0:
        return "Низкий рейтинг"
    else:
        return "Нет данных по рейтингу"

In [5]:
# Добавляем столбец с классом
joined_df['class'] = joined_df['rating'].apply(class_name)
joined_df.head()

,movieId,title,genres,userId,rating,timestamp,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,8.518667e+08,Средний рейтинг
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,9.386292e+08,Средний рейтинг
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,5.0,1.331380e+09,Высокий рейтинг
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.0,9.979383e+08,Низкий рейтинг
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,3.0,8.551901e+08,Средний рейтинг


##### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [6]:
keywords_df = pd.read_csv('F:\\Program\\git_base\\GitHub\\MyRepo\\PYDA_07\\keywords.csv', \
                         sep=',', header=0, encoding='UTF-8')
keywords_df.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [7]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [8]:
def geo_func(keywords):
    for name in geo_data:
        if keywords in geo_data[name]:
                return name
    return "undefined"
        
geo_func('хабаровск')

'Дальний Восток'

In [10]:
keywords_df['region'] = keywords_df['keyword'].apply(geo_func)
keywords_df[keywords_df['region'] != 'undefined']

,keyword,shows,region
1259,москва,153050,Центр
5858,владивосток,43038,Дальний Восток
6075,ярославль,41720,Центр
8977,мурманск,29984,Северо-Запад
9084,хабаровск,29655,Дальний Восток
11222,псков,24885,Северо-Запад
11785,тула,23898,Центр
20823,петербург,14859,Северо-Запад
21445,сахалин,14494,Дальний Восток


###### Задание 4
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
	- для каждой строки пройдите по всем годам списка years
	- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
	- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [11]:
# Повторяем выгрузку датафрейма из первой задачи
joined_df.head()

,movieId,title,genres,userId,rating,timestamp,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,8.518667e+08,Средний рейтинг
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,9.386292e+08,Средний рейтинг
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,5.0,1.331380e+09,Высокий рейтинг
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.0,9.979383e+08,Низкий рейтинг
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,3.0,8.551901e+08,Средний рейтинг


In [15]:
# Формируем годы 
years = []
for element in range(1950, 2011):
    years.append(element)  
years[:10]

[1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959]

In [16]:
# Функции простановки годов
def production_year(a):
    for year in years:
        if str(year) in a:
            return year
    return 1900
# Функции расчета среднего рейтинга
def groupby_function(joined_df):
    return joined_df.rating.mean()

In [19]:
joined_df['year'] = joined_df['title'].apply(production_year)
joined_df[joined_df['year'] != 1900].head()

,movieId,title,genres,userId,rating,timestamp,class,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,8.518667e+08,Средний рейтинг,1995
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,9.386292e+08,Средний рейтинг,1995
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,5.0,1.331380e+09,Высокий рейтинг,1995
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.0,9.979383e+08,Низкий рейтинг,1995
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,3.0,8.551901e+08,Средний рейтинг,1995


In [21]:
joined_df.groupby('year').apply(groupby_function).sort_values(ascending=False).head(5)

year
1957    4.014241
1972    4.011136
1952    4.000000
1954    3.994220
1951    3.983539
dtype: float64